In [1]:
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm as tqdm
from keras.models import load_model
from keras import backend as K
from datetime import datetime

from Dental_Tool.Inception_v3 import Custom_Inception_V3, Simple_Inception
from Dental_Tool.Inception import create_inception_v4
from Dental_Tool.Data_processing import *
from Dental_Tool.Dental_Model import *
from Dental_Tool.Process_results import *
from Dental_Tool.Dataloader import *
from Dental_Tool.KFold_v3 import *

import keras.backend as K 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import collections
import numpy as np
import keras
import time

Using TensorFlow backend.


In [2]:
# root_dir = "Results/Inception_Binary_new_Balance_Fold_520201107-003453"
# root_dir = "Results/Inception_Three_new_Balance_Fold_520201108-005152"

In [2]:
directory = [ 
                "Dental_Data/PBL/10_interdental_20200901_Max_4", 
                "Dental_Data/PBL/10_interdental_20200901_Max_4_Flip", 
                "Dental_Data/PBL/10_interdental_clahe_20200901_Max_4", 
                "Dental_Data/PBL/10_interdental_clahe_20200901_Max_4_Flip",
#                 "Dental_Data/PBL/10_interdental_20200901_Max_4_Shear",
#                 "Dental_Data/PBL/10_interdental_clahe_20200901_Max_4_Shear"
            ]

# directory = [ 
#                 "Dental_Data/PBL/10_20200901_Original", 
#                 "Dental_Data/PBL/10_20200901_Flip_Original", 
#                 "Dental_Data/PBL/10_clahe_20200901_Original", 
#                 "Dental_Data/PBL/10_clahe_20200901_Flip_Original"
#             ]

# directory = [ 
#                 "Dental_Data/PBL/10_20200901", 
#                 "Dental_Data/PBL/10_20200901_Flip", 
#                 "Dental_Data/PBL/10_clahe_20200901", 
#                 "Dental_Data/PBL/10_clahe_20200901_Flip"
#             ]

directory = [ i + "/mapping.json" for i in directory]
argscale_num = len(directory) * 20 

data = load_json(directory, interdental=True)

# dataset = json_2_dataframe_PBL(data)
dataset = json_2_dataframe_PBL_inderdental(data)

# groups = [ tooth_table for ori_src, table in dataset.groupby("ori_src") for src, teeth_table in table.groupby("source") for src_side, tooth_table in teeth_table.groupby("side") ]
# dataset = pd.concat(groups).reset_index(drop=True)

dataset

,Path,State,Class,ID,ori_src,source,tooth_num,tooth_type,side,angle
0,Dental_Data/PBL/10_interdental_20200901_Max_4/...,1,0,000408 102419 x,20200901_Max_4/08-09-15-828_000408 102419 x,20200901_Max_4/08-09-15-828_000408 102419 x_6,6,2,20200901_Max_4/08-09-15-828_000408 102419 x_6_L,-10
1,Dental_Data/PBL/10_interdental_20200901_Max_4_...,1,0,000408 102419 x,Max_4_Flip/08-09-15-828_000408 102419 x,Max_4_Flip/08-09-15-828_000408 102419 x_6,6,2,Max_4_Flip/08-09-15-828_000408 102419 x_6_L,-10
2,Dental_Data/PBL/10_interdental_clahe_20200901_...,1,0,000408 102419 x,20200901_Max_4/08-50-47-804_000408 102419 x,20200901_Max_4/08-50-47-804_000408 102419 x_6,6,2,20200901_Max_4/08-50-47-804_000408 102419 x_6_L,-10
3,Dental_Data/PBL/10_interdental_clahe_20200901_...,1,0,000408 102419 x,Max_4_Flip/08-50-47-804_000408 102419 x,Max_4_Flip/08-50-47-804_000408 102419 x_6,6,2,Max_4_Flip/08-50-47-804_000408 102419 x_6_L,-10
4,Dental_Data/PBL/10_interdental_20200901_Max_4/...,1,0,000408 102419 x,20200901_Max_4/08-09-15-844_000408 102419 x,20200901_Max_4/08-09-15-844_000408 102419 x_6,6,2,20200901_Max_4/08-09-15-844_000408 102419 x_6_R,-10
...,...,...,...,...,...,...,...,...,...,...
703835,Dental_Data/PBL/10_interdental_clahe_20200901_...,2,1,S594966_2 091718 x,4_Flip/09-28-06-956_S594966_2 091718 x,4_Flip/09-28-06-956_S594966_2 091718 x_26,26,3,4_Flip/09-28-06-956_S594966_2 091718 x_26_L,9
703836,Dental_Data/PBL/10_interdental_20200901_Max_4/...,1,0,S594966_2 091718 x,Max_4/08-50-31-686_S594966_2 091718 x,Max_4/08-50-31-686_S594966_2 091718 x_26,26,3,Max_4/08-50-31-686_S594966_2 091718 x_26_R,9
703837,Dental_Data/PBL/10_interdental_20200901_Max_4_...,1,0,S594966_2 091718 x,4_Flip/08-50-31-686_S594966_2 091718 x,4_Flip/08-50-31-686_S594966_2 091718 x_26,26,3,4_Flip/08-50-31-686_S594966_2 091718 x_26_R,9
703838,Dental_Data/PBL/10_interdental_clahe_20200901_...,1,0,S594966_2 091718 x,Max_4/09-28-06-964_S594966_2 091718 x,Max_4/09-28-06-964_S594966_2 091718 x_26,26,3,Max_4/09-28-06-964_S594966_2 091718 x_26_R,9


In [5]:
model_info = ("Inception", Inception)

performance_list = []

epochs     = 15
Fold_num = 5
num_params = 1
fold_data_num = 1

Sheng_ID = ["002555 042513 x", "000408 102419 x", "005627 120209 x", "004151 091409 x",
            "003615 010816 x", "007274 021016 x", "007274 021016 x", "004359 030716 x",
            "001742 082712 x", "002456 060517 x", "1899 120718 x", "004499 110515 x",
            "13529 092513 x",  "000411 112119 x", "008908 090309 x", "003262 103015 x",
            "10689 102418 x", "003670 020718 x", "010953 031618 x", ]

dataset_gen = K_Fold_balance_data_generator(dataset, argscale=argscale_num, k_fold_num=Fold_num, batch_size=64)
for train_dataset, valid_dataset, test_dataset, train_generator, valid_generator, test_generator in dataset_gen:
        
        train_dataset_sheng = train_dataset[train_dataset.ID.isin(Sheng_ID)]
        valid_dataset_sheng = valid_dataset[valid_dataset.ID.isin(Sheng_ID)]
        test_dataset_sheng  = test_dataset [test_dataset.ID.isin(Sheng_ID)]
        
        print(f"==================== Fold {fold_data_num} =======================")
        
        print(f"Train's # of sheng : { len(train_dataset_sheng)} ")
        print(f"Valid's # of sheng : { len(valid_dataset_sheng)} ")
        print(f"Test's # of sheng  : { len(test_dataset_sheng) } ")
        
        print(f"=================================================================")
        
        fold_data_num += 1
    

stage 0: 106240, stage 1: 196640, stage 2: 84880, stage 3: 34400
stage 0: 35360, stage 1: 65920, stage 2: 27840, stage 3: 11440
stage 0: 35520, stage 1: 65760, stage 2: 28240, stage 3: 11600
==================== Fold 1 =======================
Train's # of sheng : 22688 
Valid's # of sheng : 0 
Test's # of sheng  : 0 
stage 0: 106080, stage 1: 196800, stage 2: 84480, stage 3: 34480
stage 0: 35520, stage 1: 65760, stage 2: 28240, stage 3: 11360
stage 0: 35520, stage 1: 65760, stage 2: 28240, stage 3: 11600
==================== Fold 2 =======================
Train's # of sheng : 4026 
Valid's # of sheng : 0 
Test's # of sheng  : 18850 
stage 0: 106240, stage 1: 196640, stage 2: 84880, stage 3: 34400
stage 0: 35520, stage 1: 65760, stage 2: 28240, stage 3: 11360
stage 0: 35360, stage 1: 65920, stage 2: 27840, stage 3: 11680
==================== Fold 3 =======================
Train's # of sheng : 311 
Valid's # of sheng : 18670 
Test's # of sheng  : 3763 
stage 0: 106400, stage 1: 197440, s

In [8]:
def load_json_multitask(data_list, interdental=False):
            filter_fun = lambda x : { path: max(list(map(int, state))) for path, state in x.items() if max(list(map(int, state))) >= 0 }
               
            interdental_fun = lambda x : { path: state for path, state in x.items() }
            
            results = collections.OrderedDict()
            all_filtering_data, all_keys = [], []
            
            for dataset_path in data_list:
                        mapping_data = json.load(open(dataset_path , "r"))
                        filter_data  = filter_fun(mapping_data) if not interdental else interdental_fun(mapping_data)
                        all_filtering_data.append(filter_data)
                        all_keys.append( list(filter_data.keys()) )
            
            for keys in zip(*all_keys):
                    for key,  data in zip(keys, all_filtering_data):
                            results[key] = data[key]        
            return results

In [11]:
def json_2_dataframe_PBL_inderdental_multitask(data, mode=None):
        PBL_Columns = ["Path", "State", "Class", "bone_loss", "furcation", "buccal_furcation", "endo_lesion"]
        dataframe_L, dataframe_R  = pd.DataFrame(columns=PBL_Columns), pd.DataFrame(columns=PBL_Columns)
        data_dict_L, data_dict_R  = collections.OrderedDict(), collections.OrderedDict()   
        counter_L, counter_R = 0, 0
        
        molar = [1, 2, 3, 14, 15, 16, 17, 18, 19, 30, 31, 32]
        premolar = [ 4, 5, 12, 13, 20, 21, 28, 29 ]
        all_molar = molar + premolar
        
        less_data  = [1, 16, 17, 32]
        
        for path, info in data.items():
                state = info["state"]
                item = { 
                          "Path": path,
                          "State": state,
                          "Class": state - 1 if state >= 1 else 0,
                          **info
                }
                
                path_split = path.split("_")
                
#                 in_dir = path.split("/")[2]

                NN_IDX = 0
                for idx, i in enumerate(path_split):
                        if i == "NN":
                            NN_IDX = idx
                            break
                
                original, source = "", ""
                
                if NN_IDX == 0:
                        source = "_".join(path_split[-7:-2]) 
                        original = "_".join(path_split[-7:-3])
                
                else: 
                        source = "_".join(path_split[NN_IDX:-2])
                        original = "_".join(path_split[NN_IDX:-3])
                
                if ' ' == path_split[-4][1]:  ID = "_".join(path_split[-5:-3])
                else  : ID = path_split[-4]
                item["ID"] = ID
                
                item["ori_src"] = original
                item["source"] = source
                item["side"] = source + "_" + path[-5]
#                 if 'L' == path[-5]: item["source"] = source + '_L' 
#                 elif 'R'== path[-5]: item["source"] = source + '_R'
#                 if 
#                 else: raise ValueError
                
                item["tooth_num"] = int(path_split[-3])
                
                cond_1 = (mode == "molar"    ) and (item["tooth_num"] not in molar    )
                cond_2 = (mode == "premolar" ) and (item["tooth_num"] not in premolar )
                cond_3 = (mode == "all_molar") and (item["tooth_num"] not in all_molar)
                if cond_1 or cond_2 or cond_3 : continue
                                                  
                item["angle"] = int(path_split[-2].split(".")[0])
                
                if 'L' == path[-5]:
                        data_dict_L[counter_L] = item
                        counter_L += 1        
                
                if 'R' == path[-5]:
                        data_dict_R[counter_R] = item
                        counter_R += 1        

        
        dataframe_L = dataframe_L.from_dict(data_dict_L, "index")
        dataframe_R = dataframe_R.from_dict(data_dict_R, "index")
        
        dataframe = pd.concat([dataframe_L, dataframe_R])
        
        return dataframe

In [12]:
directory = [ 
                "Dental_Data/PBL/10_interdental_20201015_multilabel_max4", 
                "Dental_Data/PBL/10_interdental_20201015_multilabel_max4_flip", 
                "Dental_Data/PBL/10_interdental_20201015_clahe_multilabel_max4", 
                "Dental_Data/PBL/10_interdental_20201015_clahe_multilabel_max4_flip"
            ]


# directory = [ 
#                 "Dental_Data/PBL/10_20201015_multilabel", 
#                 "Dental_Data/PBL/10_20201015_multilabel_flip", 
#                 "Dental_Data/PBL/10_20201015_clahe_multilabel", 
#                 "Dental_Data/PBL/10_20201015_clahe_multilabel_flip"
#             ]

directory = [ i + "/mapping.json" for i in directory]
argscale_num = len(directory) * 20

data = load_json_multitask(directory, interdental=True)
multi_dataset = json_2_dataframe_PBL_inderdental_multitask(data)
# dataset = json_2_dataframe_PBL_Multitask(data)
multi_dataset = multi_dataset[ (multi_dataset["State"] >=0) & (multi_dataset["State"] >=0) & (multi_dataset["bone_loss"] >=0) ]

set(multi_dataset.ID) & set(Sheng_ID)

set()

In [3]:
append_match_col = lambda df: [ 1 if Class == Predict else 0  for Class, Predict in zip(df.Class, df.Predict)]
calculate_acc = lambda df: round(len(df[df.Match == 1]) / len(df) * 100, 3)

In [4]:
train_path = f"{root_dir}/Fold_1/table/train_dataset.csv"
valid_path = f"{root_dir}/Fold_1/table/valid_predict.csv"
test_path = f"{root_dir}/Fold_1/table/test_predict.csv"

train_data = pd.read_csv(train_path)
valid_data = pd.read_csv(valid_path)
test_data  = pd.read_csv(test_path)

all_data_ID = set(pd.concat([train_data, valid_data, test_data]).ID)
all_ID_dict = { pid: {"Valid": 0, "Test": 0} for pid in all_data_ID }

In [5]:
def create_correct_dict(df):
        ID_groups = df.groupby("ID")
        all_ID = set(df.ID)
        acc_dict = collections.OrderedDict()
        for ID in all_ID:
                ID_df = ID_groups.get_group(ID)
                accuracy = calculate_acc(ID_df)
                acc_dict[ID] = [len(ID_df), len(ID_df[ID_df.Match == 1]), accuracy]
        return acc_dict

In [6]:
for fold_num in range(1, 6, 1):
        valid_path = f"{root_dir}/Fold_{fold_num}/table/valid_predict.csv"
        test_path = f"{root_dir}/Fold_{fold_num}/table/test_predict.csv"
        
        valid_data = pd.read_csv(valid_path)
        test_data  = pd.read_csv(test_path)
        
        valid_data["Match"] = append_match_col(valid_data)
        test_data["Match"] = append_match_col(test_data)
        
        
        print("Valid Acc: %.2f %%"  % ( calculate_acc(valid_data) ))
        print("Test  Acc: %.2f %%"  % ( calculate_acc(test_data ) ))
        
        
        val_acc_dict  = create_correct_dict(valid_data)
        for ID, value  in val_acc_dict.items(): 
                total, match, acc = value
                all_ID_dict[ID]["Valid_Total"] = total
                all_ID_dict[ID]["Valid_Match"] = match
                all_ID_dict[ID]["Valid_Acc"]   = acc
                all_ID_dict[ID]["Valid_fold"]  = fold_num
                
                for Class in range(0, 3):
                        all_ID_dict[ID][f"Valid_Class_{Class}"] = len(valid_data[ ( valid_data.ID == ID ) & (valid_data.Class == Class) ])
               
                for Stage in range(0, 4):
                        all_ID_dict[ID][f"Valid_Stage_{Stage}"] = len(valid_data[ ( valid_data.ID == ID ) & (valid_data.State == Stage) ])
                      
            
        test_acc_dict = create_correct_dict(test_data )
        for ID, value  in test_acc_dict.items():
                total, match, acc = value
                all_ID_dict[ID]["Test_Total"] = total
                all_ID_dict[ID]["Test_Match"] = match
                all_ID_dict[ID]["Test_Acc"] = acc
                all_ID_dict[ID]["Test_fold"] = fold_num
                
                for Class in range(0, 3):
                        all_ID_dict[ID][f"Test_Class_{Class}"] = len(test_data[( test_data.ID == ID ) & (test_data.Class == Class)])
               
                for Stage in range(0, 4):
                        all_ID_dict[ID][f"Test_Stage_{Stage}"] = len(test_data[( test_data.ID == ID ) & (test_data.State == Stage)])
               
                
                
                

Valid Acc: 86.22 %
Test  Acc: 80.91 %
Valid Acc: 81.96 %
Test  Acc: 82.30 %
Valid Acc: 84.09 %
Test  Acc: 82.41 %
Valid Acc: 83.27 %
Test  Acc: 82.24 %
Valid Acc: 82.73 %
Test  Acc: 86.14 %


In [7]:
result_df = pd.DataFrame()
result_df = result_df.from_dict(all_ID_dict).T
result_df = result_df[["Valid_fold", "Valid_Total", "Valid_Match" , "Valid_Acc",
                       "Valid_Class_0", "Valid_Class_1", "Valid_Class_2",
                       "Valid_Stage_0", "Valid_Stage_1", "Valid_Stage_2", "Valid_Stage_3",
                       
                       "Test_fold" , "Test_Total" , "Test_Match"  , "Test_Acc",
                       "Test_Class_0", "Test_Class_1", "Test_Class_2",
                       "Test_Stage_0", "Test_Stage_1", "Test_Stage_2", "Test_Stage_3",
                      ]]

In [8]:
result_df = result_df.sort_values(by=['Valid_Acc', 'Test_Acc'], ascending=False)
result_df

,Valid_fold,Valid_Total,Valid_Match,Valid_Acc,Valid_Class_0,Valid_Class_1,Valid_Class_2,Valid_Stage_0,Valid_Stage_1,Valid_Stage_2,...,Test_Total,Test_Match,Test_Acc,Test_Class_0,Test_Class_1,Test_Class_2,Test_Stage_0,Test_Stage_1,Test_Stage_2,Test_Stage_3
692239 030719 x,5.0,48.0,48.0,100.000,0.0,48.0,0.0,0.0,0.0,48.0,...,48.0,48.0,100.000,0.0,48.0,0.0,0.0,0.0,48.0,0.0
701907 120318 x,2.0,300.0,300.0,100.000,300.0,0.0,0.0,300.0,0.0,0.0,...,300.0,300.0,100.000,300.0,0.0,0.0,300.0,0.0,0.0,0.0
682208 062018 x,1.0,2700.0,2697.0,99.889,2700.0,0.0,0.0,2700.0,0.0,0.0,...,2700.0,2700.0,100.000,2700.0,0.0,0.0,2700.0,0.0,0.0,0.0
544695_2 090110 x,5.0,984.0,981.0,99.695,984.0,0.0,0.0,384.0,600.0,0.0,...,908.0,895.0,98.568,908.0,0.0,0.0,368.0,540.0,0.0,0.0
544695_1 041519 x,5.0,675.0,671.0,99.407,675.0,0.0,0.0,144.0,531.0,0.0,...,728.0,708.0,97.253,728.0,0.0,0.0,138.0,590.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559666 030519 x,4.0,483.0,243.0,50.311,348.0,135.0,0.0,0.0,348.0,135.0,...,483.0,294.0,60.870,348.0,135.0,0.0,0.0,348.0,135.0,0.0
S594966_2 091718 x,5.0,393.0,197.0,50.127,57.0,336.0,0.0,48.0,9.0,336.0,...,394.0,286.0,72.589,58.0,336.0,0.0,48.0,10.0,336.0,0.0
S108236_2 022717 x,5.0,844.0,416.0,49.289,220.0,624.0,0.0,0.0,220.0,624.0,...,844.0,552.0,65.403,220.0,624.0,0.0,0.0,220.0,624.0,0.0
002555 042513 x,2.0,1800.0,835.0,46.389,486.0,1314.0,0.0,299.0,187.0,434.0,...,1800.0,696.0,38.667,486.0,1314.0,0.0,299.0,187.0,434.0,880.0


In [9]:
result_df.to_csv(f"{root_dir}/id_acc.csv")

In [10]:
weird_id = result_df[result_df.Test_fold == 1].sort_values(by=['Test_Acc'], ascending=False)
weird_id

,Valid_fold,Valid_Total,Valid_Match,Valid_Acc,Valid_Class_0,Valid_Class_1,Valid_Class_2,Valid_Stage_0,Valid_Stage_1,Valid_Stage_2,...,Test_Total,Test_Match,Test_Acc,Test_Class_0,Test_Class_1,Test_Class_2,Test_Stage_0,Test_Stage_1,Test_Stage_2,Test_Stage_3
701907 120318 x,2.0,300.0,300.0,100.000,300.0,0.0,0.0,300.0,0.0,0.0,...,300.0,300.0,100.000,300.0,0.0,0.0,300.0,0.0,0.0,0.0
514026 021611 x,2.0,748.0,717.0,95.856,748.0,0.0,0.0,715.0,33.0,0.0,...,693.0,689.0,99.423,693.0,0.0,0.0,660.0,33.0,0.0,0.0
S112552_2 043015 x,2.0,606.0,595.0,98.185,20.0,586.0,0.0,0.0,20.0,186.0,...,608.0,604.0,99.342,22.0,586.0,0.0,0.0,22.0,186.0,400.0
422254 112116 x,2.0,693.0,685.0,98.846,693.0,0.0,0.0,663.0,30.0,0.0,...,696.0,691.0,99.282,696.0,0.0,0.0,663.0,33.0,0.0,0.0
423378 092914 x,2.0,295.0,277.0,93.898,295.0,0.0,0.0,273.0,22.0,0.0,...,293.0,290.0,98.976,293.0,0.0,0.0,273.0,20.0,0.0,0.0
624137 022719 x,2.0,631.0,585.0,92.710,631.0,0.0,0.0,444.0,187.0,0.0,...,668.0,660.0,98.802,668.0,0.0,0.0,481.0,187.0,0.0,0.0
424266 101419 x,2.0,443.0,399.0,90.068,443.0,0.0,0.0,432.0,11.0,0.0,...,479.0,472.0,98.539,479.0,0.0,0.0,468.0,11.0,0.0,0.0
710456 111119 x,2.0,2852.0,2773.0,97.230,88.0,2764.0,0.0,0.0,88.0,124.0,...,2848.0,2747.0,96.454,88.0,2760.0,0.0,0.0,88.0,120.0,2640.0
422489 031815 x,2.0,754.0,684.0,90.716,694.0,60.0,0.0,540.0,154.0,60.0,...,756.0,726.0,96.032,694.0,62.0,0.0,540.0,154.0,62.0,0.0
019747_1 033011 x,2.0,725.0,648.0,89.379,725.0,0.0,0.0,648.0,77.0,0.0,...,725.0,695.0,95.862,725.0,0.0,0.0,648.0,77.0,0.0,0.0


In [11]:
weird_id.to_csv(f"{root_dir}/weird_id_acc.csv")